### Funções Auxiliares

In [1]:
def formatar(x):
    """
    Formata um valor para impressão em uma matriz.

    Parâmetros:
    x: int or str - O valor a ser formatado.

    Retorna:
    str - Valor formatado, alinhado à direita e preenchido com espaços.
    """
    # Se o valor for um número inteiro, formata como inteiro alinhado à direita com um mínimo de 3 dígitos.
    if type(x) is int:
        return f"{x:>3d}"
    # Se o valor for uma string, formata como string alinhada à direita com um mínimo de 3 caracteres.
    else:
        return f"{x:>3}"

def print_matrix(S1, S2, M):
    """
    Imprime uma matriz formatada. Pode ser útil se quisermos analisar a matriz.

    Parâmetros:
    S1: list - Lista de caracteres para rótulos de coluna.
    S2: list - Lista de caracteres para rótulos de linha.
    M: list of list - Matriz a ser impressa.

    Retorna:
    None
    """
    # Obtém a largura dos rótulos de coluna
    col_width = max(len(str(x)) for x in S1)

    # Imprime os rótulos de coluna
    print(" " * (col_width + 2) + " ".join(formatar(x) for x in S1))

    # Imprime as linhas da matriz com rótulos de linha
    for x2, linha in zip(S2, M):
        # Imprime os rótulos de linha formatados à esquerda e preenchidos com espaços, seguidos pelos valores da linha formatados.
        print("{:<{}} {}".format(x2, col_width + 1, ' '.join(map(formatar, linha))))

    # Adiciona uma linha em branco após a impressão da matriz
    print()
    
def score_subst(x1, x2, g):
    """
    Calcula a pontuação de substituição entre dois caracteres.

    Parâmetros:
    x1: str - Primeiro caracter
    x2: str - Segundo caracter
    g: int - Penalidade por gap (espaço) na substituição

    Retorna:
    int - Pontuação da substituição entre os caracteres.
    """

    # Verifica se há um gap (espaço) em pelo menos um dos caracteres
    if '-' in x1 + x2:
        return g  # Retorna a penalidade por gap

    # Verifica se os caracteres fazem match, em caso afirmativo valor= 2 (exemplo)
    if x1 == x2:
        return 2  

    # Caso contrário, retorna a penalidade por mismatch valor=-1 (exemplo)
    return -1

### Algoritmo Needleman Wunsch

In [2]:
def needleman_wunsch(S1, S2, gap_penalty, score_subst):
    """
    Algoritmo de Needleman-Wunsch para calcular o alinhamento global de duas sequências.

    Parâmetros:
    S1: str - Primeira sequência
    S2: str - Segunda sequência
    gap_penalty: int - Penalidade por espaçamento (gap)
    score_subst: função auxiliar - Função que recebe dois caracteres e retorna o score de substituição entre eles

    Retorna:
    Tuple[int, str] - Pontuação do alinhamento ótimo e a sequência alinhada
    """

    
    #Garantir que recebemos duas sequências string e que o gap penalty é inteiro.
    assert isinstance(S1, str) and isinstance(S2, str), "S1 e S2 devem ser strings."
    assert isinstance(gap_penalty, int), "A penalidade por gap deve ser um número inteiro."
    assert S1.strip() != '' and S2.strip() != '', "As sequências não podem ser vazias ou consistir apenas de espaços."
    assert all(c.isalpha() for c in S1), "A sequência S1 contém caracteres inválidos."
    assert all(c.isalpha() for c in S2), "A sequência S2 contém caracteres inválidos."

    
    
    # Adiciona um espaço '-' no início de ambas as sequências
    S2 = '-' + S2
    S1 = '-' + S1

    ncols = len(S1)
    nlins = len(S2)

    # Inicializa as matrizes de scores e de trace
    scores = [[0 for _ in range(ncols)] for _ in range(nlins)]
    trace = [[0 for _ in range(ncols)] for _ in range(nlins)]

    # Inicializa a primeira linha da matriz de scores e de trace
    scores[0] = [C * gap_penalty for C, _ in enumerate(S1)]
    trace[0] = [0 if C == 0 else 'E' for C, _ in enumerate(S1)]

    # Inicializa a primeira coluna da matriz de scores
    for L, _ in enumerate(S2):
        scores[L][0] = L * gap_penalty
        trace[L][0] = 0 if L == 0 else 'C'

    # Preenche a matriz de scores e de trace tendo em conta que as primeiras linha e coluna já foram preenchidas
    for L, (X2, linha) in enumerate(zip(S2, scores)):
        for C, (X1, V) in enumerate(zip(S1, linha)):
            if L > 0 and C > 0:
                # Calcula os valores para as três opções: diagonal, esquerda, cima
                diag = scores[L - 1][C - 1] + score_subst(X1, X2, gap_penalty)
                left = scores[L][C - 1] + gap_penalty
                up = scores[L - 1][C] + gap_penalty

                # Lista de escolhas e direções correspondentes
                choices = [diag, left, up]
                # Para conseguirmos colocar acessar às letras na matriz de trace ("DiagonalEsquerdaCima")
                directions = "DEC"

                # Encontra o máximo valor e a direção correspondente
                value = max(*choices)
                # Armazena na matriz de trace a direção escolhida para a célula com base na pontuação máxima entre as opções 
                trace[L][C] = directions[choices.index(value)]
                # Armazena na matriz de pontuações a pontuação acumulada até uma célula com base na escolha da direção que maximizou a pontuação.
                scores[L][C] = value
                
    #Se quisermos visualizar as matrizes (de score e de trace)
    print_matrix(S1, S2, scores)
    print_matrix(S1, S2, trace)
    
    # Retorna a pontuação total e a sequência alinhada com recurso à função de reconstrução do alinhamento
    return scores[-1][-1], reconstruct_alignment(S1, S2, trace)


def reconstruct_alignment(S1, S2, trace):
    """
    Reconstrói a sequência alinhada a partir da matriz de trace.

    Parâmetros:
    S1: str - Primeira sequência
    S2: str - Segunda sequência
    trace: list - Matriz de rastreamento

    Retorna:
    Tuple[str, str] - Par de sequências alinhadas
    """

    aligned_seq1 = ""
    aligned_seq2 = ""
    L, C = len(S2) - 1, len(S1) - 1

    # Realiza o trace-back até atingir a primeira célula da matriz
    while trace[L][C] != 0:
        if trace[L][C] == 'D':
            aligned_seq1 = S1[C] + aligned_seq1
            aligned_seq2 = S2[L] + aligned_seq2
            L -= 1
            C -= 1
        elif trace[L][C] == 'E':
            aligned_seq1 = S1[C] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            C -= 1
        elif trace[L][C] == 'C':
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = S2[L] + aligned_seq2
            L -= 1

    # Retorna o melhor par de sequências alinhadas
    return aligned_seq1, aligned_seq2

In [3]:
needleman_wunsch("ACGTA", "CCAT", -2, score_subst)

     -   A   C   G   T   A
-    0  -2  -4  -6  -8 -10
C   -2  -1   0  -2  -4  -6
C   -4  -3   1  -1  -3  -5
A   -6  -2  -1   0  -2  -1
T   -8  -4  -3  -2   2   0

     -   A   C   G   T   A
-    0   E   E   E   E   E
C    C   D   D   E   E   E
C    C   D   D   D   D   D
A    C   D   C   D   D   D
T    C   C   D   D   D   E



(0, ('ACGTA', 'CCAT-'))

### Função para calcular a sequência consenso

In [17]:
def consenso(alinhamento: list[str]) -> str:
    # Inicializa uma string vazia para armazenar o consenso
    res = ""
    
    # Itera sobre as colunas do alinhamento usando zip(*alinhamento)
    for x in zip(*alinhamento):
        # Chama a função consenso_char para obter o consenso da coluna atual e concatena ao resultado
        res += consenso_char(x)
        
        # Imprime o resultado parcial (opcional, para visualização)
        #print(res)
    
    # Retorna a sequência consenso
    return res

# Define uma função para obter o segundo elemento de uma tupla
def get_second_element(tupla):
    return tupla[1]

def consenso_char(lista: list[str]) -> str:
    # Concatena os caracteres da lista em uma string e remove os espaços ("-")
    string = "".join(lista).replace("-", "")
    
    # Cria uma lista de tuplas com o caractere e sua contagem, ordenada pela contagem em ordem decrescente
    ocorrencias = sorted([(x, string.count(x)) for x in set(string)], key=get_second_element, reverse=True)
    
    #Para verificarmos a contagem (opcional)
    print(ocorrencias)
    
    # Retorna o caractere mais frequente na coluna (primeiro caractere da primeira tupla)
    return list(ocorrencias)[0][0]

In [18]:
consenso(["ACGTA","CCAT-"])

[('A', 1), ('C', 1)]
[('C', 2)]
[('A', 1), ('G', 1)]
[('T', 2)]
[('A', 1)]


'ACATA'


### Função para resconstruir o alinhamento 

A função **constroi_alin** realiza a construção de um novo alinhamento inserindo espaços nas posições onde há espaços no alinhamento do consenso. Para fazer isso, ela realiza os seguintes passos:

**Cálculo do Consenso:**

- Chama a função consenso(alinhamento) para calcular o consenso das sequências presentes no alinhamento original. O consenso é uma sequência de caracteres que representa, para cada posição, o caractere mais frequente naquela posição entre as sequências do alinhamento.

**Alinhamento Global:**

- Utiliza a função needleman_wunsch para realizar um alinhamento global entre o consenso (seq_cons) e a sequência seq. O resultado do alinhamento é uma pontuação (score) e um par de sequências alinhadas (A1 e A2).


**Identificação de Espaços no Alinhamento do Consenso:**

- Encontra as posições onde há espaços na sequência A1 (que representa o alinhamento do consenso) e armazena essas posições na lista pos.

**Adição de Espaços ao Alinhamento Original:**

- Chama a função add_spaces(alinhamento, pos) para adicionar espaços ao alinhamento original nas posições identificadas.

**Inclusão da Sequência Alinhada ao Novo Alinhamento:**

- Adiciona a sequência alinhada A2 (a sequência da sequência seq após o alinhamento) ao final do novo alinhamento.

**Retorno do Novo Alinhamento:**

- Retorna o novo alinhamento, que consiste nas sequências do alinhamento original com os espaços adicionados nas posições correspondentes ao alinhamento do consenso, juntamente com a sequência alinhada A2.

In [6]:
def constroi_alin(alinhamento: list[str], seq: str) -> list[str]:
    # Calcula o consenso do alinhamento
    seq_cons = consenso(alinhamento)
    
    # Executa o alinhamento global entre o consenso e a sequência dada
    score, (A1, A2) = needleman_wunsch(seq_cons, seq, -4, score_subst)
    
    # Encontra as posições onde há espaços no alinhamento do consenso
    pos = [p for p, x in enumerate(A1) if x == "-"]
    
    # Adiciona espaços ao alinhamento original nas posições identificadas
    return add_spaces(alinhamento, pos) + [A2]


def add_spaces(alinhamento: list[str], pos: list[int]) -> list[str]:
    # Inicializa uma nova lista para armazenar o alinhamento modificado
    new_align = []
    
    # Itera sobre as sequências do alinhamento original
    for seq in alinhamento:
        # Adiciona espaços nas posições identificadas
        for p in pos:
            seq = seq[:p] + "-" + seq[p:]
        # Adiciona a sequência modificada à nova lista
        new_align.append(seq)
    
    # Retorna a nova lista de alinhamento
    return new_align

In [7]:
constroi_alin(["A-C","ACT"], "ACCTG")

A
AC
ACC
     -   A   C   C
-    0  -4  -8 -12
A   -4   2  -2  -6
C   -8  -2   4   0
C  -12  -6   0   6
T  -16 -10  -4   2
G  -20 -14  -8  -2

     -   A   C   C
-    0   E   E   E
A    C   D   E   E
C    C   C   D   D
C    C   C   D   D
T    C   C   C   C
G    C   C   C   C



['A-C--', 'ACT--', 'ACCTG']

### Função incluindo todas as anteriores para realizar o alinhamento progressivo

In [8]:
def alinhamento_progressivo(seqs: list[str]) -> list[str]:
    # Desempacota as duas primeiras sequências e armazena o restante em 'resto'
    A1, A2, *resto = seqs
    
    # Realiza um alinhamento global entre as duas primeiras sequências usando a função NW
    score, (A1, A2) = needleman_wunsch(A1, A2, -4, score_subst)
    
    # Inicializa uma lista 'alinhamento' com as sequências alinhadas
    alinhamento = [A1, A2]
    
    # Itera sobre as sequências restantes para construir o alinhamento progressivo
    for seq in resto:
        # Chama a função constroi_alin para adicionar a sequência ao alinhamento progressivo
        alinhamento = constroi_alin(alinhamento, seq)
        
    # Retorna o alinhamento 
    return alinhamento

In [14]:
alinhamento_progressivo("ACGTA CCAT ACCA".split())

     -   A   C   G   T   A
-    0  -4  -8 -12 -16 -20
C   -4  -1  -2  -6 -10 -14
C   -8  -5   1  -3  -7 -11
A  -12  -6  -3   0  -4  -5
T  -16 -10  -7  -4   2  -2

     -   A   C   G   T   A
-    0   E   E   E   E   E
C    C   D   D   E   E   E
C    C   D   D   D   D   D
A    C   D   C   D   D   D
T    C   C   D   D   D   E

A
AC
ACA
ACAT
ACATA
     -   A   C   A   T   A
-    0  -4  -8 -12 -16 -20
A   -4   2  -2  -6 -10 -14
C   -8  -2   4   0  -4  -8
C  -12  -6   0   3  -1  -5
A  -16 -10  -4   2   2   1

     -   A   C   A   T   A
-    0   E   E   E   E   E
A    C   D   E   D   E   D
C    C   C   D   E   E   E
C    C   C   D   D   D   D
A    C   D   C   D   D   D



['ACGTA', 'CCAT-', 'AC-CA']